In [1]:
import numpy as np
import pandas as pd
import scipy.io
import h5py

In [2]:
# downloaded from https://data.mendeley.com/datasets/p7ft2bvphx/1
v1 = scipy.io.loadmat('data/empirical/SPKcounts_V20160929cue_MW_250X250ms.mat')
v2 = scipy.io.loadmat('data/empirical/SPKcounts_V20160930cue_MW_250X250ms.mat')
v3 = scipy.io.loadmat('data/empirical/SPKcounts_V20161005cue_MW_250X250ms.mat')
v4 = scipy.io.loadmat('data/empirical/SPKcounts_V20161017cue_MW_250X250ms.mat')
w1 = scipy.io.loadmat('data/empirical/SPKcounts_W20160112cue_MW_250X250ms.mat')
w2 = scipy.io.loadmat('data/empirical/SPKcounts_W20160113cue_MW_250X250ms.mat')
w3 = scipy.io.loadmat('data/empirical/SPKcounts_W20160121cue_MW_250X250ms.mat')
w4 = scipy.io.loadmat('data/empirical/SPKcounts_W20160122cue_MW_250X250ms.mat')
vs = [v1, v2, v3, v3]
ws = [w1, w2, w3, w4]

In [3]:
dfs = []
columns = ['monkey', 'session', 'block_type', 'trial', 'left', 'right', 'correct', 'reversal_at_trial', 'monkey_accuracy']

for monkey in ['V', 'W']:
    files = vs if monkey=='V' else ws
    for f, file in enumerate(files):
        print(f)
        df = pd.DataFrame(columns=columns)
        data = file['Y']
        bid = data[:,7]
        block = data[:,9]
        trial = data[:,5]
        letter = data[:,0]
        location = data[:,1]
        best_chosen = data[:,4]
        block_completed = data[:,12]
        # reversal_at_trial = data[:,6]
        idxs = np.argwhere(data[:,6]==1)[:,0]
        reversal_at_trial = {}
        for idx in idxs:
            bb = bid[idx]
            tt = trial[idx]
            reversal_at_trial[bb]=tt
        # print(idxs, reversal_at_trial)
        # raise
        # reversal_at_trial = []
        # for bb in np.unique(bid):
        #     idx = np.argwhere(data[:,6]==1)[bb-1][0]
        #     reversal_at_trial.append(trial[idx])
        # print(reversal_at_trial)
        left = []
        right = []
        correct = []
        for row in range(data.shape[0]):
            if location[row]==0 and letter[row]==0:
                left.append('A')
                right.append('B')
            if location[row]==0 and letter[row]==1:
                left.append('B')
                right.append('A')
            if location[row]==1 and letter[row]==0:
                left.append('B')
                right.append('A')
            if location[row]==1 and letter[row]==1:
                left.append('A')
                right.append('B')
            # if block[row] == 2:  # where
            if best_chosen[row]==1 and location[row]==0:
                correct.append('left')
            if best_chosen[row]==1 and location[row]==1:
                correct.append('right')
            if best_chosen[row]==0 and location[row]==0:
                correct.append('right')
            if best_chosen[row]==0 and location[row]==1:
                correct.append('left')
            # if block[row] == 1:  # what
            #     if best_chosen[row]==1 and letter[row]==0:
            #         correct.append('left')
            #     if best_chosen[row]==1 and letter[row]==1:
            #         correct.append('right')
            #     if best_chosen[row]==0 and letter[row]==0:
            #         correct.append('right')
            #     if best_chosen[row]==0 and letter[row]==1:
            #         correct.append('left')
            # print(f"block {block[row]}, left {left[-1]}, right {right[-1]}, chose {best_chosen[row]}, correct {correct[-1]}") 
        df['block'] = bid
        df['block_type'] = block
        df['trial'] = trial
        df['left'] = left
        df['right'] = right
        df['correct'] = correct
        df['monkey_choice'] = location
        df['monkey_accuracy'] = best_chosen
        df['block_completed'] = block_completed
        df = df[df['block_completed'] != 0]
        temp = []
        for bb in df['block'].unique():
            for t in df.query("block==@bb")['trial'].unique():
                temp.append(reversal_at_trial[bb])
            # print(bid, t)
        df['reversal_at_trial'] = temp
        df['monkey'] = [monkey for _ in range(df.shape[0])]
        df['session'] = [f for _ in range(df.shape[0])]
        dfs.append(df)

behavioral_data = pd.concat(dfs, ignore_index=True)
# behavioral_data['left'] = behavioral_data['letter'].replace('A', -1)
# behavioral_data['left'] = behavioral_data['letter'].replace('B', 1)
# behavioral_data['right'] = behavioral_data['location'].replace('A', -1)
# behavioral_data['right'] = behavioral_data['location'].replace('B', 1)
behavioral_data['block_type'] = behavioral_data['block_type'].replace(1, 'what')
behavioral_data['block_type'] = behavioral_data['block_type'].replace(2, 'where')
behavioral_data['monkey_choice'] = behavioral_data['monkey_choice'].replace(1, -1)
behavioral_data['monkey_choice'] = behavioral_data['monkey_choice'].replace(0, 1)

# behavioral_data = behavioral_data[behavioral_data['block_completed'] != 0]
behavioral_data = behavioral_data.drop(columns='block_completed')

0
1
2
3
0
1
2
3


In [4]:
behavioral_data.query("monkey=='V' & block==10")[20:60]

,monkey,session,block_type,trial,left,right,correct,reversal_at_trial,monkey_accuracy,block,monkey_choice
740,V,0,what,21,A,B,left,50,1,10,1
741,V,0,what,22,A,B,left,50,1,10,1
742,V,0,what,23,B,A,right,50,1,10,-1
743,V,0,what,24,B,A,right,50,1,10,-1
744,V,0,what,25,A,B,left,50,1,10,1
745,V,0,what,26,B,A,right,50,1,10,-1
746,V,0,what,27,B,A,right,50,1,10,-1
747,V,0,what,28,A,B,left,50,1,10,1
748,V,0,what,29,B,A,right,50,1,10,-1
749,V,0,what,30,A,B,left,50,1,10,1


In [5]:
behavioral_data.to_pickle("data/empirical.pkl")

In [13]:
def check_opposite(row):
    return (row['left'] == 'A' and row['right'] == 'B') or \
           (row['left'] == 'B' and row['right'] == 'A')

# Apply the check to all rows
all_opposite = behavioral_data.apply(check_opposite, axis=1).all()

print("All rows contain opposite letters:", all_opposite)

All rows contain opposite letters: True


# Spikes

In [23]:
# downloaded from https://data.mendeley.com/datasets/p7ft2bvphx/1
v1 = h5py.File('data/emp_spikes/RastVect-V20160929-binsize10ms-align2SaccadeOnset.mat')
v2 = h5py.File('data/emp_spikes/RastVect-V20160930-binsize10ms-align2SaccadeOnset.mat')
v3 = h5py.File('data/emp_spikes/RastVect-V20161005-binsize10ms-align2SaccadeOnset.mat')
v4 = h5py.File('data/emp_spikes/RastVect-V20161017-binsize10ms-align2SaccadeOnset.mat')
w1 = h5py.File('data/emp_spikes/RastVect-W20160112-binsize10ms-align2SaccadeOnset.mat')
w2 = h5py.File('data/emp_spikes/RastVect-W20160113-binsize10ms-align2SaccadeOnset.mat')
w3 = h5py.File('data/emp_spikes/RastVect-W20160121-binsize10ms-align2SaccadeOnset.mat')
w4 = h5py.File('data/emp_spikes/RastVect-W20160122-binsize10ms-align2SaccadeOnset.mat')
vs2 = [v1, v2, v3, v4]
ws2 = [w1, w2, w3, w4]

In [42]:
# t_start, t_end = 145, 150  # 50ms
# t_start, t_end = 125, 150  # 250ms
t_start, t_end = 150, 160  # 100ms
for monkey in ['V', 'W']:
    files = vs if monkey=='V' else ws
    files2 = vs2 if monkey=='V' else ws2
    for session in [0,1,2,3]:
        print(f"monkey {monkey} session {session}")
        trial_data = files[session]['Y']
        bid = trial_data[:,7]
        tid = trial_data[:,5]
        spk = files2[session]['aligned2event']
        for t in range(len(tid)):
            block = bid[t]
            trial = tid[t]
            spikes = spk[t]
            summed = np.sum(spikes[:,t_start:t_end], axis=1)
            file = f"data/emp_spikes/monkey{monkey}_session{session}_block{block}_trial{trial}_spikes_post100ms.npz"
            np.savez_compressed(file, spikes=summed)

monkey V session 0
monkey V session 1
monkey V session 2
monkey V session 3
monkey W session 0
monkey W session 1
monkey W session 2
monkey W session 3


In [27]:
emp = pd.read_pickle("data/empirical.pkl")
for monkey in ['V', 'W']:
    for session in [0,1,2,3]:
        spike_sum = 0
        block_sum = 0
        trial_sum = 0
        for block in emp.query("monkey==@monkey & session==@session")['block'].unique():
            block_sum += 1
            for trial in emp.query("monkey==@monkey & session==@session & block==@block")['trial'].unique():
                file = f"data/emp_spikes/monkey{monkey}_session{session}_block{block}_trial{trial}_spikes.npz"
                spikes = np.load(file)['spikes']
                spike_sum += spikes.sum()
                trial_sum += 1
        print(f"{monkey} {session}, trials {trial_sum}, spikes {spike_sum}")

V 0, trials 1920, spikes 355428.0
V 1, trials 1920, spikes 586290.0
V 2, trials 1920, spikes 497784.0
V 3, trials 1920, spikes 250853.0
W 0, trials 1840, spikes 270388.0
W 1, trials 1920, spikes 282342.0
W 2, trials 1840, spikes 240406.0
W 3, trials 1920, spikes 203081.0


# RL

In [40]:
# columns = ['monkey', 'session', 'block', 'trial', 'va', 'vb', 'vl', 'vr', 'w', 'al', 'ar', 'dw', 'dvc', 'dvs', 'dva'] 
columns = ['monkey', 'session', 'block', 'trial'] 
labels = [
    "DeltaRel",
    "omega",
    "omega_BlockMean",
    "DeltaOmega",
    "pLeft",
    "pChosen",
    "V_Stim_C",
    "V_Act_C",
    "UpdatedV_Stim_C",
    "UpdatedV_Act_C",
    "RPE_Stim",
    "RPE_Action",
    "RPE_combined",
    "V_Stim_U",
    "V_Act_U",
    "V_Stim1",
    "V_Stim2",
    "V_Act1",
    "V_Act2",
    "choseBetterStim",
    "choseBetterAct",
    "V_StimR",
    "V_StimL",
    "V_Act1_End",
    "V_Act2_End",
    "V_StimR_End",
    "V_StimL_End",
    "V_StimPref",
    "V_StimUnpref",
    "DV_cho",
    "DVcho_Stim",
    "DVcho_Action",
    "DV_left",
    "DV_right",
    "EffOmega",
    "beta_Stim_t",
    "beta_Action_t",
    "prevChoseBetterStim",
    "prevChoseBetterAct",
    "DeltaV_Stim",
    "DeltaV_Act",
    "DeltaV_Stim_End",
    "DeltaV_Act_End"]
for monkey in ['V', 'W']:
    for session in range(4):
        print(f"monkey {monkey} session {session}")
        sess = 1+session if monkey=='V' else 1+session+4
        files = vs if monkey=='V' else ws
        trial_data = files[session]['Y']
        bid = trial_data[:,7]
        tid = trial_data[:,5]
        letter = trial_data[:,0]
        location = trial_data[:,1]
        rl_vars = scipy.io.loadmat(f'data/matlab/all_vars_sess{sess}.mat')['AllVars'][0][0]
        for t in range(len(tid)):
            block = bid[t]
            trial = tid[t]
            df_list = [monkey, session, block, trial]
            for v, var in enumerate(rl_vars):
                try:
                    value = var.flatten()[t]
                except:
                    print(f"skipping {monkey} {session} {block} {trial} {labels[v]}")
                    value = np.nan
                df_list.append(value)                    
            file = f"data/emp_spikes/monkey{monkey}_session{session}_block{block}_trial{trial}_values.pkl"
            df = pd.DataFrame([df_list], columns=columns+labels)
            df.to_pickle(file)

monkey V session 0
skipping V 0 19 25 omega_BlockMean
skipping V 0 19 26 omega_BlockMean
skipping V 0 19 27 omega_BlockMean
skipping V 0 19 28 omega_BlockMean
skipping V 0 19 29 omega_BlockMean
skipping V 0 19 30 omega_BlockMean
skipping V 0 19 31 omega_BlockMean
skipping V 0 19 32 omega_BlockMean
skipping V 0 19 33 omega_BlockMean
skipping V 0 19 34 omega_BlockMean
skipping V 0 19 35 omega_BlockMean
skipping V 0 19 36 omega_BlockMean
skipping V 0 19 37 omega_BlockMean
skipping V 0 19 38 omega_BlockMean
skipping V 0 19 39 omega_BlockMean
skipping V 0 19 40 omega_BlockMean
skipping V 0 19 41 omega_BlockMean
skipping V 0 19 42 omega_BlockMean
skipping V 0 19 43 omega_BlockMean
skipping V 0 19 44 omega_BlockMean
skipping V 0 19 45 omega_BlockMean
skipping V 0 19 46 omega_BlockMean
skipping V 0 19 47 omega_BlockMean
skipping V 0 19 48 omega_BlockMean
skipping V 0 19 49 omega_BlockMean
skipping V 0 19 50 omega_BlockMean
skipping V 0 19 51 omega_BlockMean
skipping V 0 19 52 omega_BlockMean
s